# Train and Eval

The purpose of this Notebook is to train CNN models on our dataset and evaluate their performance on the classification task. The deeper analysis of the patterns present in their results and how they compare to the survey takers will take place in the “Results Analysis” Notebook.

The Notebook is divided in two main sections: “Training” and “Evaluation” that can be run independently. Both sections, however, require the “Imports and Setup” section to be run beforehand.

## Expected File Structure

The code expects the data to be in a folder called “Images”, present at the same level as the Notebook. Inside said folder, the images should be divided between three folders “train”, “valid” and “test”, and inside each of those they should be divided between two folders called “Main” and “Supporting”, according to their class.

There should also be a folder called “Checkpoints”, again at the same level as the Notebook, where the model will save its checkpoints during the training process, as well as its history JSON file when it finishes.

## Imports and Setup

In [ ]:
import os
import numpy as np
import tarfile
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import get_file
import tensorflow.keras.optimizers
#from tensorflow.keras import activations

AUTOTUNE = tf.data.experimental.AUTOTUNE

# This Notebook was made using tensorflow version 2.1.0,
# other versions may work, but there's no guarantee of that.
print(tf.__version__)

In [ ]:
# Source for these methods: 
# https://subscription.packtpub.com/book/data/9781838829131/2/ch02lvl1sec14/implementing-resnet-from-scratch

# Loads an image and its label in the format to be used by tensorflow
# Also applies the preprocessing methods with which we achieved our best results
# Receives the image path and the target size, which is, by default, 
# set as the size of the iamges downloaded from MAL
def load_image_and_label(image_path, target_size=(225, 350)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)
    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == NARRATIVE_CLASSES)  # One-hot encode.
    label = tf.dtypes.cast(label, tf.float32)
    return image, label

# Loads and prepares the selected dataset to be used by tensorflow
# Must be called once for each subset (train, valid, test)
# Receives a pattern representing the paths of the images in the subset,
# examples found in the following section, and if the data should or not be suffled,
# which should only need to be used for the training subset
def prepare_dataset(data_pattern, shuffle=False):
    dataset = (tf.data.Dataset.list_files(data_pattern).map(load_image_and_label,
               num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE))
    
    if shuffle:
        dataset = dataset.shuffle(BUFFER_SIZE)
        
    return dataset.prefetch(BATCH_SIZE)

NARRATIVE_CLASSES = ["Main", "Supporting"]

# Change the batch size according to how much VRAM you have available
BATCH_SIZE = 2
BUFFER_SIZE = 16

## Training

In [ ]:
# Patterns representing the paths of the images in the train and valid subsets
train_pattern = os.path.sep.join(
    ["Images", 'train', '*', '*.png'])
valid_pattern = os.path.sep.join(
    ["Images", 'valid', '*', '*.png'])

# Loading the subsets
train_dataset = prepare_dataset(train_pattern, 
                                shuffle=True)
valid_dataset = prepare_dataset(valid_pattern)

In [ ]:
# Creating, compuling and training the model

import tensorflow.keras.applications

# Size of the iamges downloaded from MAL
array_input = Input(shape=(225, 350, 3))

# Loading the ResNet model from tensorflow, trained on the iamgenet dataset
resModel = tf.keras.applications.ResNet50(include_top=False,
                                          weights="imagenet",
                                          input_tensor=array_input,
                                          input_shape=(225, 350, 3),
                                          pooling='avg')

# Use this if you only want to train our new layers
#for layer in resModel.layers[:143]:
#    layer.trainable = False

# Adding our classification layers to the end of the model
model = Sequential()
model.add(resModel)
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

# Compiling the model with the combination of parameters we used to achieve 
# our best results
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=2e-6),
              metrics=['accuracy'])

# Saving a checkpoint whenever we achieve a new best result during training
model_checkpoint_callback = ModelCheckpoint(
    filepath='./Checkpoints/ResNet50 rmsprop.{epoch:02d}-{val_accuracy:.2f}.hdf5',
    save_weights_only=False,
    monitor='val_accuracy',
    save_best_only=True)

# Training the model
EPOCHS = 100
hist = model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=EPOCHS,
          callbacks=[model_checkpoint_callback])

In [ ]:
# Saving the final state of the model
model.save('./Checkpoints/ResNet50 rmsprop '+EPOCHS+'.hdf5')

### Visualising the training history

In [ ]:
# Plotting accuracy over time

import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='best')
plt.show()

In [ ]:
# Plotting loss over time

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='best')
plt.show()

In [ ]:
# Saving the training history as a JSON file

import json

histDict = hist.history

newDict = {}
for key in histDict:
    newDict[key] = []
    for value in histDict[key]:
        newDict[key].append(float(value))

json.dump(newDict, open('./Checkpoints/ResNet50 rmsprop '+EPOCHS+' Hist.json', 'w'))

## Evaluation

In [ ]:
# Loading an example checkpoint
model = load_model('./Checkpoints/ResNet50 rmsprop.15-0.62.hdf5')

In [ ]:
# Loading the test subset

test_pattern = os.path.sep.join(
    ["Images", 'test', '*', '*.png'])

test_dataset = prepare_dataset(test_pattern)

In [ ]:
# Running the evaluation

result = model.evaluate(test_dataset)
print(f'Test accuracy: {result[1]}')

In [ ]:
# Loads and predicts a single image
# Return both the result of the prediction and the target label
def testImage(folder, group, role, name):
    image, label = load_image_and_label(os.path.sep.join(
                    [folder, group, role, name]))
    image = np.expand_dims(image, axis=0)
    result = model.predict(image)
    return result, label

In [ ]:
# Example of how to use testImage

name = "Hildegard von Mariendorf.png"
result, label = testImage("Images", "test", "Main", name)
print(label)
print(result)

In [ ]:
# Storing the predictions for each character in the test subset

import pathlib

characters = {"Main": {}, "Supporting": {}}

for role in ["Main", "Supporting"]:
    for path in pathlib.Path("Images/test/"+role).iterdir():
        name = str(path).split("\\")[-1]
        result, label = testImage("Images", "test", role, name)
        characters[role][name] = result

In [ ]:
# Converting the results and saving them as a JSON file

import json

newDict = {"Main": {}, "Supporting": {}}
for role in ["Main", "Supporting"]:
    for character in characters[role]:
        newDict[role][character] = []
        for value in characters[role][character]:
            newDict[role][character].append(float(value))

json.dump(newDict, open("Model Test Results.json", "w"))